In [1]:
#The csv file containing the music_info.csv and created cluster labels imported as a data frame through Pandas
import pandas as pd

In [2]:
threeclusters_df = pd.read_csv("3clusters_musicinfo.csv")
threeclusters_df.head(5)

,Unnamed: 0,name,artist,danceability,energy,key,mode,valence,tempo,Cluster
0,0,Mr. Brightside,The Killers,0.355,0.918,1,1,0.240,0.619996,1
1,1,Wonderwall,Oasis,0.409,0.892,2,1,0.651,0.730137,1
2,2,Come as You Are,Nirvana,0.508,0.826,4,0,0.543,0.502363,1
3,3,Take Me Out,Franz Ferdinand,0.279,0.664,9,1,0.490,0.437682,2
4,4,Creep,Radiohead,0.515,0.430,7,1,0.104,0.384441,0


In [3]:
threeclusters_df['X_vector'] = [[threeclusters_df.danceability.iloc[i],threeclusters_df.energy.iloc[i],threeclusters_df.key.iloc[i],threeclusters_df['mode'].iloc[i],threeclusters_df.valence.iloc[i],threeclusters_df.tempo.iloc[i]] for i in range(0, len(threeclusters_df))]
threeclusters_df.head(5)

,Unnamed: 0,name,artist,danceability,energy,key,mode,valence,tempo,Cluster,X_vector
0,0,Mr. Brightside,The Killers,0.355,0.918,1,1,0.240,0.619996,1,"[0.355, 0.918, 1, 1, 0.24, 0.6199962326545135]"
1,1,Wonderwall,Oasis,0.409,0.892,2,1,0.651,0.730137,1,"[0.409, 0.892, 2, 1, 0.651, 0.7301366709223718]"
2,2,Come as You Are,Nirvana,0.508,0.826,4,0,0.543,0.502363,1,"[0.508, 0.826, 4, 0, 0.543, 0.5023629628079282]"
3,3,Take Me Out,Franz Ferdinand,0.279,0.664,9,1,0.490,0.437682,2,"[0.279, 0.664, 9, 1, 0.49, 0.4376818267439671]"
4,4,Creep,Radiohead,0.515,0.430,7,1,0.104,0.384441,0,"[0.515, 0.43, 7, 1, 0.104, 0.3844408631407104]"


In [4]:
#The below comes from following along with the Kaggle notebook found here: https://www.kaggle.com/code/prashantdandriyal/online-svm-classifier-sgdclassifier

In [4]:
#import the necessary library, modules, functions, settings 
import numpy as np 
import random
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [5]:
#Define the input and target variable
X = threeclusters_df['X_vector']
y = threeclusters_df['Cluster']

In [6]:
# Break the data up into 4 arrays
X_split = np.array_split(X, 4)
y_split = np.array_split(y, 4)

/opt/anaconda3/envs/erdos_may_2024_v2/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


In [7]:
# Convert to lists of values
X_split0 = X_split[0].tolist()
X_split1 = X_split[1].tolist()
X_split2 = X_split[2].tolist()
X_split3 = X_split[3].tolist()
y_split0 = y_split[0].tolist()
y_split1 = y_split[1].tolist()
y_split2 = y_split[2].tolist()
y_split3 = y_split[3].tolist()

In [8]:
# Define the scaler
scaler = StandardScaler()

In [9]:
#define functions that create batches and then shuffle those batches
def create_batches(iterable_X, iterable_y, batch_number, batch_size):
    X_batches = []
    y_batches = []
    for i in range(batch_number):
        X_batches.append(iterable_X[i*batch_size:(i+1)*batch_size])
        y_batches.append(iterable_y[i*batch_size:(i+1)*batch_size])
    return X_batches, y_batches
    

def shuffle_batch(iterable_X, iterable_y,shuffle_number):
    indices = [i for i in shuffle_number]
    random.shuffle(indices)
    shuffledX = [iterable_X[i] for i in indices]
    shuffledy = [iterable_y[i] for i in indices]
    return shuffledX, shuffledy

In [10]:
# Define the model that accepts the scaled features (training data), 
# a 'firstRun' flag to distinguish between the first run-through and online training, and trains an SGD classifier 
# the function returns a loaded model

def SGD_MODClassifier(scaledData_X, scaledData_Y, firstRun=0, model=None):
    
    try:
        # for a reproducible result
        np.random.seed(123)
        #HyperParameter and model element assigning
        xTrain = scaledData_X
        yTrain = scaledData_Y


        # Make sure model is only created once and trained numerously
        if(firstRun==1):
            model =  SGDClassifier(loss="hinge", penalty="l2", alpha=0.0001, max_iter=3000, tol=None, shuffle=True, verbose=0, learning_rate='adaptive', eta0=0.01, early_stopping=False)
            # Suggested number of passes for convergence
            model.n_iter = np.ceil(10**6 / len(yTrain))
            # Train model using fit
            model.fit(xTrain, yTrain)
        else:
            batch_size = 500
            batch_number = int(np.ceil(len(xTrain) / float(batch_size)))
            X_train_batches, y_train_batches = create_batches(xTrain,yTrain, batch_number, batch_size)
            # Use the model passed to the function
            # we repeat partial fit for all shuffled batches.
            for _ in range(3):
                for chunk_X, chunk_y in zip(X_train_batches, y_train_batches):
                    shuffle_number = range(len(chunk_X))
                    chunk_X, chunk_y = shuffle_batch(chunk_X, chunk_y, shuffle_number)
                    model.partial_fit(chunk_X, chunk_y)
            
        
        print('Done Training')
        message = "Successfully trained SGD_MODClassifier"

    except Exception as e:
        message = e
        model = None

    return message, model

In [11]:
# Part 1: fit for first time
X_train, X_test, y_train, y_test = train_test_split(X_split0,y_split0,random_state=123)

# Scale the data to be between -1 and 1
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# run the model and evaluate
message, model = SGD_MODClassifier(X_train, y_train, 1); print(message)
model.score(X_test, y_test)

Done Training
Successfully trained SGD_MODClassifier


0.9460227272727273

In [12]:
#Save model
import pickle
modelSaveFile = 'model_data_id.sav'
pickle.dump(model, open(modelSaveFile, 'wb'))

In [13]:
# Part 2: partial_fit
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_split1,y_split1,random_state=123)

# Load the model
model = pickle.load(open(modelSaveFile, 'rb'))

# Scale the data to be between -1 and 1
scaler.fit(X_train1)
X_train1 = scaler.transform(X_train1)
X_test1 = scaler.transform(X_test1)

message, model = SGD_MODClassifier(X_train1, y_train1, 0, model); print(message)
print(model.score(X_test1, y_test1))

# Save model
modelSaveFile = 'model_data_id2.sav'
pickle.dump(model, open(modelSaveFile, 'wb'))

Done Training
Successfully trained SGD_MODClassifier
0.9731691919191919


In [14]:
# Part 3: partial_fit
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_split2,y_split2,random_state=123)

# Load the model
model = pickle.load(open(modelSaveFile, 'rb'))
# Scale the data to be between -1 and 1
scaler.fit(X_train2)
X_train2 = scaler.transform(X_train2)
X_test2 = scaler.transform(X_test2)

message, model = SGD_MODClassifier(X_train2, y_train2, 0, model); print(message)
print(model.score(X_test2, y_test2))

# Save model
modelSaveFile = 'model_data_id3.sav'
pickle.dump(model, open(modelSaveFile, 'wb'))

Done Training
Successfully trained SGD_MODClassifier
0.9390782828282829


In [15]:
# Part 4: partial_fit
X_train3, X_test3, y_train3, y_test3 = train_test_split(X_split3,y_split3,random_state=123)

# Load the model
model = pickle.load(open(modelSaveFile, 'rb'))
# Scale the data to be between -1 and 1
scaler.fit(X_train3)
X_train3 = scaler.transform(X_train3)
X_test3 = scaler.transform(X_test3)

message, model = SGD_MODClassifier(X_train3, y_train3, 0, model); print(message)
print(model.score(X_test3, y_test3))

# Save model
modelSaveFile = 'model_data_id4.sav'
pickle.dump(model, open(modelSaveFile, 'wb'))

Done Training
Successfully trained SGD_MODClassifier
0.9384469696969697


In [ ]:
#evaluation of the previous model iterations 

In [16]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

def metrics(yActual, yPredicted):
    print(f'accuracy_score: {accuracy_score(yActual, yPredicted)}')
    print(f"f1 score: {f1_score(yActual, yPredicted, average='weighted')}")
    print(f"precision score: {precision_score(yActual, yPredicted, average='weighted')}")
    print(f"recall score: {recall_score(yActual, yPredicted, average='weighted')}")

In [17]:
metrics(y_test, model.predict(X_test))

accuracy_score: 0.9179292929292929
f1 score: 0.9159685748659036
precision score: 0.9325569287208335
recall score: 0.9179292929292929


In [18]:
metrics(y_test1, model.predict(X_test1))

accuracy_score: 0.9176136363636364
f1 score: 0.9138808315305065
precision score: 0.9271395691693096
recall score: 0.9176136363636364


In [19]:
metrics(y_test2, model.predict(X_test2))

accuracy_score: 0.9232954545454546
f1 score: 0.9209866792681215
precision score: 0.9345759897658288
recall score: 0.9232954545454546


In [20]:
metrics(y_test3, model.predict(X_test3))

accuracy_score: 0.9384469696969697
f1 score: 0.9363272705233328
precision score: 0.9435476027909792
recall score: 0.9384469696969697


In [ ]:
#The results were very similar to the other notebook and the danger of overfitting still looms. 